In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Hello World: Il Vostro Primo Circuito Quantistico

Costruite uno [stato di Bell](https://en.wikipedia.org/wiki/Bell_state) (due qubit entangled) ed eseguitelo in tre modi:

1. **Simulazione ideale** — risultati perfetti, nessun account richiesto
2. **Simulazione con rumore** — simula un dispositivo reale, nessun account richiesto
3. **Hardware quantistico reale** — richiede un [account IBM Quantum](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account)

## Costruire il circuito

In [ ]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

qc.draw(output="mpl")

## Opzione 1: Simulazione ideale (nessun account richiesto)
Utilizza `StatevectorSampler` — un simulatore locale con risultati perfetti e privi di rumore.

In [ ]:
from qiskit.primitives import StatevectorSampler

result = StatevectorSampler().run([qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(counts)

## Opzione 2: Simulazione con rumore (nessun account richiesto)
Utilizza `FakeManilaV2` — un simulatore locale che imita un vero dispositivo quantistico IBM, incluse le sue caratteristiche di rumore. Il circuito deve prima essere transpilato (adattato) al set di gate e alla connettività dei qubit del dispositivo.

In [ ]:
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

backend = FakeManilaV2()
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## Opzione 3: Hardware quantistico reale
Richiede un account IBM Quantum. Consultate [Setting up your IBM Quantum account](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account) per i dettagli.

Se non avete ancora salvato le credenziali in questa sessione Binder, eseguite prima questo:

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
    token="<your-api-key>",
    instance="<your-crn>",
    overwrite=True
)

**Nota:** I job su hardware reale possono richiedere del tempo a seconda dei tempi di coda. Se la cella è ancora in esecuzione, potete controllare lo stato del vostro job e vedere i risultati su [quantum.cloud.ibm.com/workloads](https://quantum.cloud.ibm.com/workloads?user=me).

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
print(f"Running on {backend.name}")

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## Cosa fare dopo?
- **[Tutorials](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=docs/tutorials)** — guide passo-passo su algoritmi, mitigazione degli errori, transpilazione e altro
- **[Courses](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=learning/courses)** — percorsi di apprendimento strutturati dalle basi quantistiche al calcolo su scala utility
- **[Local testing mode](https://janlahmann.github.io/Qiskit-documentation/#no-token-use-local-testing-mode)** — eseguite la maggior parte dei notebook senza un account IBM Quantum